# Automated ML

In [1]:
import datetime
import json
import pathlib
import pprint
import requests

import joblib
from sklearn.metrics import classification_report

In [2]:
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails

In [3]:
from nd00333.model.automl.train import run_config as automl_train_config
from nd00333.model.deploy import run_config as deploy_config
from nd00333 import utils as package_utils

In [4]:
print("azureml.core.VERSION", azureml.core.VERSION)

azureml.core.VERSION 1.21.0


## Create an experiment

Create an experiment identified by the creation date. The purpose of identifying the experiments is to not mix the manually run experiments (using this jupyter notebook) with the experiments run using the deployment automation of the master git branch.


In [5]:
#!az logout

In [5]:
workspace = Workspace.from_config()
utcnow = datetime.datetime.utcnow().strftime("%Y-%m-%d-%H-%m")
utcnow = "2021-01-29-09-02"
experiment_name = f"jupyter-{utcnow}"

{"message": "Found the config file in: /app/config.json"}
{"message": "Found the config file in: /app/config.json"}
{"message": "Found the config file in: /app/config.json"}
{"message": "Found the config file in: /app/config.json"}
{"message": "Failed to load or parse file /home/vscode/.azureml/auth/azureProfile.json. It will be overridden by default settings."}
{"message": "Failed to load or parse file /home/vscode/.azureml/auth/azureProfile.json. It will be overridden by default settings."}
{"message": "Failed to load or parse file /home/vscode/.azureml/auth/azureProfile.json. It will be overridden by default settings."}
{"message": "Failed to load or parse file /home/vscode/.azureml/auth/azureProfile.json. It will be overridden by default settings."}
{"message": "Failed to load or parse file /home/vscode/.azureml/auth/az.json. It will be overridden by default settings."}
{"message": "Failed to load or parse file /home/vscode/.azureml/auth/az.json. It will be overridden by default se

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DWREUYXJ2 to authenticate.


{"message": "55b7f2d4-c8e3-430a-973c-c62c58b35fc0 - TokenRequest:Getting a token via device code"}
{"message": "55b7f2d4-c8e3-430a-973c-c62c58b35fc0 - TokenRequest:Getting a token via device code"}
{"message": "55b7f2d4-c8e3-430a-973c-c62c58b35fc0 - TokenRequest:Getting a token via device code"}
{"message": "55b7f2d4-c8e3-430a-973c-c62c58b35fc0 - TokenRequest:Getting a token via device code"}


You have logged in. Now let us find all the subscriptions to which you have access...


{"message": "b991fd3f-0ccb-44f7-b848-447764223503 - CacheDriver:Acquiring new access token from MRRT token."}
{"message": "b991fd3f-0ccb-44f7-b848-447764223503 - CacheDriver:Acquiring new access token from MRRT token."}
{"message": "b991fd3f-0ccb-44f7-b848-447764223503 - CacheDriver:Acquiring new access token from MRRT token."}
{"message": "b991fd3f-0ccb-44f7-b848-447764223503 - CacheDriver:Acquiring new access token from MRRT token."}
{"message": "b991fd3f-0ccb-44f7-b848-447764223503 - TokenRequest:Getting a new token from a refresh token"}
{"message": "b991fd3f-0ccb-44f7-b848-447764223503 - TokenRequest:Getting a new token from a refresh token"}
{"message": "b991fd3f-0ccb-44f7-b848-447764223503 - TokenRequest:Getting a new token from a refresh token"}
{"message": "b991fd3f-0ccb-44f7-b848-447764223503 - TokenRequest:Getting a new token from a refresh token"}
{"message": "b991fd3f-0ccb-44f7-b848-447764223503 - CacheDriver:Returning token derived from mrrt refresh."}
{"message": "b991fd

Interactive authentication successfully completed.


In [6]:
experiment = Experiment(workspace, experiment_name)
print(f"experiment_name {experiment_name}")

{"message": "Created a worker pool for first use"}
{"message": "Created a worker pool for first use"}
{"message": "Created a worker pool for first use"}
{"message": "Created a worker pool for first use"}


experiment_name jupyter-2021-01-29-09-02


## Dataset

Load the dataset from local files on disk into the default AzureML workspace datastore.
Store the versioned (by the "_1" suffix), combined train+validate dataset for the purpose of cross-validation, and test sets under separate datastore paths. The uploading of large datasets tends to break with "The write operation timed out" when using the Python SDK and therefore the `az` is used. See https://docs.microsoft.com/en-us/answers/questions/43980/cannot-upload-local-files-to-azureml-datastore-pyt.html

In [7]:
dataset_trainandvalidate_name = "ids2018trainandvalidate_1"
dataset_test_name = "ids2018test_1"
dataset_2017_name = "ids2017full_1"

In [8]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code D9ATJRDZL to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "660b3398-b80e-49d2-bc5b-ac1dc93b5254",
    "id": "a24a24d5-8d87-4c8a-99b6-91ed2d2df51f",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Udacity CloudLabs Sub - 09",
    "state": "Enabled",
    "tenantId": "660b3398-b80e-49d2-bc5b-ac1dc93b5254",
    "user": {
      "name": "odl_user_136633@udacitylabs.onmicrosoft.com",
      "type": "user"
    }
  }
]


In [9]:
!az extension add --name azure-cli-ml --version 1.21.0

- Installing ...

In [11]:
!az --version

azure-cli                         2.18.0

core                              2.18.0
telemetry                          1.0.6

Extensions:
azure-cli-ml                      1.21.0

Python location '/opt/az/bin/python3'
Extensions directory '/home/vscode/.azure/cliextensions'

Python (Linux) 3.6.10 (default, Jan 15 2021, 09:54:35) 
[GCC 8.3.0]

Legal docs and information: aka.ms/AzureCliLegal


Your CLI is up-to-date.

Please let us know how we are doing: https://aka.ms/azureclihats
and let us know if you're interested in trying out our newest features: https://aka.ms/CLIUXstudy


In [10]:
%%time
!az ml datastore upload --name workspaceblobstore --verbose \
    --src-path datasets/ids2018trainandvalidate --target-path \
    $dataset_trainandvalidate_name

Found the config file in: /app/config.json
Uploading an estimated of 1 files
Uploading datasets/ids2018trainandvalidate/data.csv
Uploaded datasets/ids2018trainandvalidate/data.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Command ran in 900.310 seconds (init: 0.385, invoke: 899.925)
CPU times: user 54.3 s, sys: 10.5 s, total: 1min 4s
Wall time: 15min 1s


In [11]:
%%time
!az ml datastore upload --name workspaceblobstore --verbose \
    --src-path datasets/ids2018test --target-path \
    $dataset_test_name

Found the config file in: /app/config.json
Uploading an estimated of 1 files
Uploading datasets/ids2018test/data.csv
Uploaded datasets/ids2018test/data.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Command ran in 527.825 seconds (init: 0.411, invoke: 527.414)
CPU times: user 30.3 s, sys: 5.69 s, total: 36 s
Wall time: 8min 49s


In [12]:
%%time
!az ml datastore upload --name workspaceblobstore --verbose \
    --src-path datasets/ids2017full --target-path \
    $dataset_2017_name

Found the config file in: /app/config.json
Uploading an estimated of 8 files
Uploading datasets/ids2017full/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
Uploaded datasets/ids2017full/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv, 1 files out of an estimated total of 8
Uploading datasets/ids2017full/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Uploaded datasets/ids2017full/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv, 2 files out of an estimated total of 8
Uploading datasets/ids2017full/Friday-WorkingHours-Morning.pcap_ISCX.csv
Uploaded datasets/ids2017full/Friday-WorkingHours-Morning.pcap_ISCX.csv, 3 files out of an estimated total of 8
Uploading datasets/ids2017full/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
Uploaded datasets/ids2017full/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv, 4 files out of an estimated total of 8
Uploading datasets/ids2017full/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Uploaded datasets/ids2017full/

Upload and register the uploaded datasets into AzureML workspace. In principle the above `az` commands should not be necessary.

In [13]:
%%time
!PYTHONPATH=. python nd00333/dataset/register/register.py \
    --dataset-path datasets --dataset-name ids2018trainandvalidate --dataset-version 1 \
    --dataset-type tabular

{"message": "Found the config file in: /app/config.json"}
{"message": "Found the config file in: /app/config.json"}
{"message": "<azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f52db02cfd0>"}
{"message": "datastore.upload_files", "kwargs": {"files": ["/app/datasets/ids2018trainandvalidate/data.csv"], "target_path": "ids2018trainandvalidate_1", "overwrite": false}}
{"message": "Called AzureBlobDatastore.upload_files"}
Uploading an estimated of 1 files
{"message": "Uploading an estimated of 1 files"}
Target already exists. Skipping upload for ids2018trainandvalidate_1/data.csv
{"message": "Target already exists. Skipping upload for ids2018trainandvalidate_1/data.csv"}
Uploaded 0 files
{"message": "Uploaded 0 files"}
{"message": "Finished AzureBlobDatastore.upload with count=0."}
{"message": "TabularDatasetFactory.from_delimited_files", "kwargs": {"path": ["<azureml.data.datapath.DataPath object at 0x7f52dafe8b50>"], "set_column_types": {"Flow Duration": "<azureml

In [14]:
%%time
!PYTHONPATH=. python nd00333/dataset/register/register.py \
     --dataset-path datasets --dataset-name ids2018test --dataset-version 1 \
     --dataset-type tabular

{"message": "Found the config file in: /app/config.json"}
{"message": "Found the config file in: /app/config.json"}
{"message": "<azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f065e4ec250>"}
{"message": "datastore.upload_files", "kwargs": {"files": ["/app/datasets/ids2018test/data.csv"], "target_path": "ids2018test_1", "overwrite": false}}
{"message": "Called AzureBlobDatastore.upload_files"}
Uploading an estimated of 1 files
{"message": "Uploading an estimated of 1 files"}
Target already exists. Skipping upload for ids2018test_1/data.csv
{"message": "Target already exists. Skipping upload for ids2018test_1/data.csv"}
Uploaded 0 files
{"message": "Uploaded 0 files"}
{"message": "Finished AzureBlobDatastore.upload with count=0."}
{"message": "TabularDatasetFactory.from_delimited_files", "kwargs": {"path": ["<azureml.data.datapath.DataPath object at 0x7f065e506150>"], "set_column_types": {"Flow Duration": "<azureml.data.dataset_factory.DataType object at 0x7f066

In [15]:
%%time
!PYTHONPATH=. python nd00333/dataset/register/register.py \
     --dataset-path datasets --dataset-name ids2017full --dataset-version 1 \
     --dataset-type tabular

{"message": "Found the config file in: /app/config.json"}
{"message": "Found the config file in: /app/config.json"}
{"message": "<azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f81d1ba5390>"}
{"message": "datastore.upload_files", "kwargs": {"files": ["/app/datasets/ids2017full/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv", "/app/datasets/ids2017full/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv", "/app/datasets/ids2017full/Friday-WorkingHours-Morning.pcap_ISCX.csv", "/app/datasets/ids2017full/Monday-WorkingHours.pcap_ISCX.csv", "/app/datasets/ids2017full/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv", "/app/datasets/ids2017full/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv", "/app/datasets/ids2017full/Tuesday-WorkingHours.pcap_ISCX.csv", "/app/datasets/ids2017full/Wednesday-workingHours.pcap_ISCX.csv"], "target_path": "ids2017full_1", "overwrite": false}}
{"message": "Called AzureBlobDatastore.upload_files"}
Uploading an estima

## AutoML Configuration

It should be noted that several settings that may affect the model performance are used in AutoMLConfig in order to speed up the training:
- the number of `iterations` (individual model runs) is limited to 15,
- instead of a cross-validation method, a single split into the training and validation data is specified using `validation_size`=0.3 in order to reduce the training time,
- the slow `enable_stack_ensemble` ensemble model is excluded,
- only the "LightGBM", "LogisticRegression", "SGD", "XGBoostClassifier" models are allowed in the runs in `allowed_models`. Models from the "RandomForest" family unbounded by `max_depth` may grow very large (to e.g. several hundreds Mbytes).

The individual model runs are performed multi-threaded using all cores available on the compute instance by `max_cores_per_iteration=`-1. The number of `max_concurrent_iterations` is set to the number of the nodes in the compute cluster.

See [nd00333/model/automl/train/run_config.py](nd00333/model/automl/train/run_config.py) for more details.

The `norm_macro_recall` is used as the performance metrics. See [README.md](README.md) for the rationale.

Perform the configuration of the AutoML run, including setting up the remote AML compute cluster

In [16]:
%%time
automl_run_config = automl_train_config.main(
                    dataset_trainandvalidate_name=dataset_trainandvalidate_name)

{"message": "Found the config file in: /app/config.json"}
{"message": "Found the config file in: /app/config.json"}
{"message": "Found the config file in: /app/config.json"}
{"message": "Found the config file in: /app/config.json"}


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


{"message": "main", "status": {"currentNodeCount": 0, "targetNodeCount": 0, "nodeStateCounts": {"preparingNodeCount": 0, "runningNodeCount": 0, "idleNodeCount": 0, "unusableNodeCount": 0, "leavingNodeCount": 0, "preemptedNodeCount": 0}, "allocationState": "Steady", "allocationStateTransitionTime": "2021-01-30T19:03:01.056000+00:00", "errors": null, "creationTime": "2021-01-30T19:02:55.373586+00:00", "modifiedTime": "2021-01-30T19:03:10.789141+00:00", "provisioningState": "Succeeded", "provisioningStateTransitionTime": null, "scaleSettings": {"minNodeCount": 0, "maxNodeCount": 5, "nodeIdleTimeBeforeScaleDown": "PT2400S"}, "vmPriority": "LowPriority", "vmSize": "STANDARD_D12_V2"}}
{"message": "main", "status": {"currentNodeCount": 0, "targetNodeCount": 0, "nodeStateCounts": {"preparingNodeCount": 0, "runningNodeCount": 0, "idleNodeCount": 0, "unusableNodeCount": 0, "leavingNodeCount": 0, "preemptedNodeCount": 0}, "allocationState": "Steady", "allocationStateTransitionTime": "2021-01-30T1

{"message": "main", "compute_target": {"id": "/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourceGroups/aml-quickstarts-136633/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-136633/computes/starts-ws-136633", "name": "starts-ws-136633", "location": "southcentralus", "tags": null, "identity": null, "properties": {"description": null, "resourceId": null, "computeType": "AmlCompute", "computeLocation": "southcentralus", "provisioningState": "Succeeded", "provisioningErrors": null, "properties": {"vmSize": "STANDARD_D12_V2", "vmPriority": "LowPriority", "scaleSettings": {"minNodeCount": 0, "maxNodeCount": 5, "nodeIdleTimeBeforeScaleDown": "PT2400S"}, "userAccountCredentials": {"adminUserName": null, "adminUserPassword": null, "adminUserSshPublicKey": null}, "subnet": {"id": null}, "remoteLoginPortPublicAccess": "Enabled"}, "status": {"currentNodeCount": 0, "targetNodeCount": 0, "nodeStateCounts": {"preparingNodeCount": 0, "runningNodeCount": 0, "idleNodeCou

CPU times: user 726 ms, sys: 67.4 ms, total: 793 ms
Wall time: 23.9 s


Submit the AutoML configuration to the compute cluster

In [17]:
%%time
automl_run = experiment.submit(config=automl_run_config, show_output=False)

Running on remote.
CPU times: user 1.2 s, sys: 141 ms, total: 1.34 s
Wall time: 13.3 s


In [18]:
%%time
automl_run.wait_for_completion(show_output=True)


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+-----------------

{"message": "f5366610-eee7-4f88-a3ac-adb164e5e4cc - CacheDriver:Cached token is expired at 2021-01-30 19:28:35.099456.  Refreshing"}
{"message": "f5366610-eee7-4f88-a3ac-adb164e5e4cc - CacheDriver:Cached token is expired at 2021-01-30 19:28:35.099456.  Refreshing"}
{"message": "f5366610-eee7-4f88-a3ac-adb164e5e4cc - CacheDriver:Cached token is expired at 2021-01-30 19:28:35.099456.  Refreshing"}
{"message": "f5366610-eee7-4f88-a3ac-adb164e5e4cc - CacheDriver:Cached token is expired at 2021-01-30 19:28:35.099456.  Refreshing"}
{"message": "f5366610-eee7-4f88-a3ac-adb164e5e4cc - TokenRequest:Getting a new token from a refresh token"}
{"message": "f5366610-eee7-4f88-a3ac-adb164e5e4cc - TokenRequest:Getting a new token from a refresh token"}
{"message": "f5366610-eee7-4f88-a3ac-adb164e5e4cc - TokenRequest:Getting a new token from a refresh token"}
{"message": "f5366610-eee7-4f88-a3ac-adb164e5e4cc - TokenRequest:Getting a new token from a refresh token"}
{"message": "f5366610-eee7-4f88-a3ac

         2   MaxAbsScaler SGD                               0:07:37       0.5349    0.5349
         3   StandardScalerWrapper XGBoostClassifier        0:09:19       0.7074    0.7074
         0   MaxAbsScaler LightGBM                          0:09:45       0.4484    0.7074
         6   StandardScalerWrapper LightGBM                 0:04:33       0.0000    0.7074
         7   MaxAbsScaler LightGBM                          0:04:16       0.5254    0.7074
         1   MaxAbsScaler XGBoostClassifier                 0:38:11       0.8223    0.8223
         9   StandardScalerWrapper LightGBM                 0:09:45       0.8152    0.8223
         8   StandardScalerWrapper LogisticRegression       0:17:00       0.7859    0.8223
        12   MaxAbsScaler LightGBM                          0:05:47       0.7015    0.8223


{"message": "c8f63365-ff3c-45fd-84af-c087f1eac2cd - CacheDriver:Cached token is expired at 2021-01-30 20:23:39.743347.  Refreshing"}
{"message": "c8f63365-ff3c-45fd-84af-c087f1eac2cd - CacheDriver:Cached token is expired at 2021-01-30 20:23:39.743347.  Refreshing"}
{"message": "c8f63365-ff3c-45fd-84af-c087f1eac2cd - CacheDriver:Cached token is expired at 2021-01-30 20:23:39.743347.  Refreshing"}
{"message": "c8f63365-ff3c-45fd-84af-c087f1eac2cd - CacheDriver:Cached token is expired at 2021-01-30 20:23:39.743347.  Refreshing"}
{"message": "c8f63365-ff3c-45fd-84af-c087f1eac2cd - TokenRequest:Getting a new token from a refresh token"}
{"message": "c8f63365-ff3c-45fd-84af-c087f1eac2cd - TokenRequest:Getting a new token from a refresh token"}
{"message": "c8f63365-ff3c-45fd-84af-c087f1eac2cd - TokenRequest:Getting a new token from a refresh token"}
{"message": "c8f63365-ff3c-45fd-84af-c087f1eac2cd - TokenRequest:Getting a new token from a refresh token"}
{"message": "c8f63365-ff3c-45fd-84af

        10   StandardScalerWrapper LightGBM                 0:20:18       0.8163    0.8223
         4   StandardScalerWrapper XGBoostClassifier        1:10:48       0.8225    0.8225
        13   StandardScalerWrapper LogisticRegression       0:26:42       0.3787    0.8225
        11   MaxAbsScaler LogisticRegression                0:43:08          nan    0.8225
ERROR: {
    "additional_properties": {},
    "error": {
        "additional_properties": {
            "debugInfo": null
        },
        "code": "UserError",
        "severity": null,
        "message": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_format": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_parameters": {},
        "reference_code": null,
        "details_uri": null,
        "target": null,
        "details": [],
        "inner_error": {
            "additional_properties": {},
            "code": "Resource

{"message": "018ee35a-3d54-4858-a927-705f8dba60d2 - CacheDriver:Cached token is expired at 2021-01-30 21:18:43.240201.  Refreshing"}
{"message": "018ee35a-3d54-4858-a927-705f8dba60d2 - CacheDriver:Cached token is expired at 2021-01-30 21:18:43.240201.  Refreshing"}
{"message": "018ee35a-3d54-4858-a927-705f8dba60d2 - CacheDriver:Cached token is expired at 2021-01-30 21:18:43.240201.  Refreshing"}
{"message": "018ee35a-3d54-4858-a927-705f8dba60d2 - CacheDriver:Cached token is expired at 2021-01-30 21:18:43.240201.  Refreshing"}
{"message": "018ee35a-3d54-4858-a927-705f8dba60d2 - TokenRequest:Getting a new token from a refresh token"}
{"message": "018ee35a-3d54-4858-a927-705f8dba60d2 - TokenRequest:Getting a new token from a refresh token"}
{"message": "018ee35a-3d54-4858-a927-705f8dba60d2 - TokenRequest:Getting a new token from a refresh token"}
{"message": "018ee35a-3d54-4858-a927-705f8dba60d2 - TokenRequest:Getting a new token from a refresh token"}
{"message": "018ee35a-3d54-4858-a927

        14    VotingEnsemble                                0:28:37       0.8347    0.8347
CPU times: user 1min 41s, sys: 6.45 s, total: 1min 47s
Wall time: 2h 15min 21s


{'runId': 'AutoML_ae1d998c-ec87-447d-bf70-328476f80063',
 'target': 'starts-ws-136633',
 'status': 'Completed',
 'startTimeUtc': '2021-01-30T19:03:45.434125Z',
 'endTimeUtc': '2021-01-30T21:18:15.618462Z',
 'properties': {'num_iterations': '15',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'norm_macro_recall',
  'train_split': '0.3',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'starts-ws-136633',
  'AMLSettingsJsonString': '{"path":null,"name":"jupyter-2021-01-29-09-02","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-136633","workspace_name":"quick-starts-ws-136633","region":"southcentralus","compute_target":"starts-ws-136633","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":15,"primary_metric":"norm_macro_recall","task_type":"classification","data_script":null,"validation_size":0.3,"n_cross_validations":null,"

In [20]:
assert(automl_run.get_status() == "Completed")

## Run Details

The widget below shows the first few best runs of the experiment

In [21]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model


Retrieve the best run

In [22]:
best_run = package_utils.get_best_run(experiment, automl_run)

In [23]:
print(best_run)
print(best_run.get_metrics())

Run(Experiment: jupyter-2021-01-29-09-02,
Id: AutoML_ae1d998c-ec87-447d-bf70-328476f80063_14,
Type: azureml.scriptrun,
Status: Completed)
{'average_precision_score_macro': 0.8632478824794572, 'precision_score_micro': 0.9729835328927925, 'f1_score_weighted': 0.9724378086065742, 'weighted_accuracy': 0.9902775001227279, 'f1_score_micro': 0.9729835328927925, 'norm_macro_recall': 0.8346544651286354, 'average_precision_score_micro': 0.9937617318918045, 'AUC_macro': 0.9867022926602761, 'AUC_micro': 0.9995343721531104, 'precision_score_weighted': 0.9730670628193983, 'AUC_weighted': 0.9882813889438005, 'f1_score_macro': 0.8641743907763539, 'recall_score_macro': 0.84646486047659, 'log_loss': 0.22456691217636446, 'accuracy': 0.9729835328927925, 'recall_score_micro': 0.9729835328927925, 'recall_score_weighted': 0.9729835328927925, 'matthews_correlation': 0.9060454227132824, 'balanced_accuracy': 0.84646486047659, 'precision_score_macro': 0.9091641765283224, 'average_precision_score_weighted': 0.978

Save the best model locally

In [24]:
print(best_run.get_details())

{'runId': 'AutoML_ae1d998c-ec87-447d-bf70-328476f80063_14', 'target': 'starts-ws-136633', 'status': 'Completed', 'startTimeUtc': '2021-01-30T20:49:34.62987Z', 'endTimeUtc': '2021-01-30T21:18:11.219349Z', 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'norm_macro_recall\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'jupyter-2021-01-29-09-02\',\'compute_target\':\'starts-ws-136633\',\'subscription_id\':\'a24a24d5-8d87-4c8a-99b6-91ed2d2df51f\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_ae1d998c-ec87-447d-bf70-328476f80063_14","experiment_name":"jupyter-2021-01-29-09-02","workspace_name":"quick-starts-ws-136633","subscription_i

In [25]:
print(best_run.get_properties())

{'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'norm_macro_recall\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'jupyter-2021-01-29-09-02\',\'compute_target\':\'starts-ws-136633\',\'subscription_id\':\'a24a24d5-8d87-4c8a-99b6-91ed2d2df51f\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_ae1d998c-ec87-447d-bf70-328476f80063_14","experiment_name":"jupyter-2021-01-29-09-02","workspace_name":"quick-starts-ws-136633","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group_name":"aml-quickstarts-136633"}}]}', 'training_percent': '100', 'predicted_cost': None, 'iteration': '14', '_aml_system_scenario_identification': 'Remote.Child'

In [26]:
%%time
best_run.download_file("outputs/model.pkl", "models/automl/model.pkl")

CPU times: user 1.35 s, sys: 719 ms, total: 2.07 s
Wall time: 28.2 s


In [27]:
print("model size in Bytes", pathlib.Path("models/automl/model.pkl").stat().st_size)

model size in Bytes 13767058


Download other files from the best_run

In [28]:
print(best_run.get_file_names())

['accuracy_table', 'automl_driver.py', 'azureml-logs/55_azureml-execution-tvmps_fb457a87b5a3e48452dcc5a2beafe8fe1e6cd84aff0c08302b4299be59eed3c5_p.txt', 'azureml-logs/65_job_prep-tvmps_fb457a87b5a3e48452dcc5a2beafe8fe1e6cd84aff0c08302b4299be59eed3c5_p.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_fb457a87b5a3e48452dcc5a2beafe8fe1e6cd84aff0c08302b4299be59eed3c5_p.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'confusion_matrix', 'logs/azureml/108_azureml.log', 'logs/azureml/azureml_automl.log', 'logs/azureml/dataprep/python_span_51f4d14a-00da-4251-866c-5c0cb116123c.jsonl', 'logs/azureml/dataprep/python_span_63a54182-da6e-47db-8ab6-6038f8923ee9.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/conda_env_v_1_0_0.yml', 'outputs/env_dependencies.json', 'outputs/model.pkl', 'outputs/pipeline_graph.json', 'outputs/scoring_file_v_1_0_0.py']


In [29]:
best_run.download_file("outputs/conda_env_v_1_0_0.yml",
                       "models/automl/conda_env_v_1_0_0.yml")
best_run.download_file("outputs/scoring_file_v_1_0_0.py",
                       "models/automl/scoring_file_v_1_0_0.py")
best_run.download_file("outputs/env_dependencies.json",
                       "models/automl/env_dependencies.json")

best_run.download_file("automl_driver.py",
                       "models/automl/automl_driver.py")
best_run.download_file("logs/azureml/azureml_automl.log",
                       "models/automl/azureml_automl.log")
best_run.download_file("azureml-logs/70_driver_log.txt",
                       "models/automl/70_driver_log.txt")

Explore the model

In [30]:
%%time
if 0:  # This method works only for AutoML
    _, fitted_model = automl_run.get_output()

CPU times: user 10 µs, sys: 2 µs, total: 12 µs
Wall time: 21.7 µs


In [31]:
%%time
fitted_model = joblib.load("models/automl/model.pkl")

CPU times: user 2.01 s, sys: 336 ms, total: 2.35 s
Wall time: 2.61 s


In [32]:
info = {}

for key, value in fitted_model.get_params()["Pipeline"].items():
    if key == "prefittedsoftvotingclassifier__weights":
        info[key] = value
    if key == "prefittedsoftvotingclassifier__estimators":
        info[key] = []
        for estimator in value:
            constitute_model = estimator[1].steps[-1][-1]
            info[key].append(constitute_model)

pprint.pprint(info)


{'prefittedsoftvotingclassifier__estimators': [XGBoostClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                  colsample_bynode=1, colsample_bytree=1, eta=0.001, gamma=0,
                  learning_rate=0.1, max_delta_step=0, max_depth=10,
                  max_leaves=15, min_child_weight=1, missing=nan,
                  n_estimators=100, n_jobs=-1, nthread=None,
                  objective='multi:softprob', random_state=0,
                  reg_alpha=2.0833333333333335, reg_lambda=0.9375,
                  scale_pos_weight=1, seed=None, silent=None, subsample=0.9,
                  tree_method='auto', verbose=-10, verbosity=0),
                                               XGBoostClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                  colsample_bynode=1, colsample_bytree=1, gamma=0,
                  learning_rate=0.1, max_delta_step=0, max_depth=3,
                  min_child_weight=1, missing=nan, n_estimators=100, n_jobs=-1,


## Model testing

Test the model on the test set from 2018 and on an additional out-of-sample test set from 2017.

Test the model on the 2018 dataset

In [33]:
%%time
test = Dataset.get_by_name(
            workspace=workspace,
            name=dataset_test_name,
        ).to_pandas_dataframe()
x_test, y_test = test.drop(labels=["Label"], axis=1), test["Label"]

CPU times: user 23.5 s, sys: 17.5 s, total: 41 s
Wall time: 2min 13s


In [34]:
%%time
y_test_predict = fitted_model.predict(x_test)

CPU times: user 1h 38min 33s, sys: 6min 16s, total: 1h 44min 49s
Wall time: 42min 50s


In [35]:
%%time
cr = classification_report(digits=4,
                           y_true=y_test,
                           y_pred=y_test_predict,
                           output_dict=False)
print(cr)

{"message": "9b60fd54-1820-45ae-a594-6136f217df34 - CacheDriver:Cached token is expired at 2021-01-30 22:13:50.025400.  Refreshing"}
{"message": "9b60fd54-1820-45ae-a594-6136f217df34 - CacheDriver:Cached token is expired at 2021-01-30 22:13:50.025400.  Refreshing"}
{"message": "9b60fd54-1820-45ae-a594-6136f217df34 - CacheDriver:Cached token is expired at 2021-01-30 22:13:50.025400.  Refreshing"}
{"message": "9b60fd54-1820-45ae-a594-6136f217df34 - CacheDriver:Cached token is expired at 2021-01-30 22:13:50.025400.  Refreshing"}
{"message": "9b60fd54-1820-45ae-a594-6136f217df34 - TokenRequest:Getting a new token from a refresh token"}
{"message": "9b60fd54-1820-45ae-a594-6136f217df34 - TokenRequest:Getting a new token from a refresh token"}
{"message": "9b60fd54-1820-45ae-a594-6136f217df34 - TokenRequest:Getting a new token from a refresh token"}
{"message": "9b60fd54-1820-45ae-a594-6136f217df34 - TokenRequest:Getting a new token from a refresh token"}
{"message": "9b60fd54-1820-45ae-a594

                          precision    recall  f1-score   support

                  Benign     0.9904    0.9908    0.9906   2048549
                     Bot     0.9992    0.9998    0.9995     43419
        Brute Force -Web     0.9877    0.9524    0.9697        84
        Brute Force -XSS     1.0000    0.9756    0.9877        41
        DDOS attack-HOIC     0.9999    1.0000    0.9999     48951
  DDoS attacks-LOIC-HTTP     1.0000    1.0000    1.0000     86440
   DoS attacks-GoldenEye     0.9924    0.9994    0.9959      8184
        DoS attacks-Hulk     0.9974    1.0000    0.9987      7658
DoS attacks-SlowHTTPTest     0.7663    0.5213    0.6205     41976
   DoS attacks-Slowloris     0.9789    1.0000    0.9893      2598
          FTP-BruteForce     0.7186    0.8854    0.7934     58010
           Infilteration     0.3417    0.3291    0.3353     29297
           SQL Injection     1.0000    0.4615    0.6316        13
          SSH-Bruteforce     0.9988    0.9998    0.9993     56208

        

In [36]:
del test

Test the model on the 2017 dataset

In [50]:
%%time
test_2017 = Dataset.get_by_name(
            workspace=workspace,
            name=dataset_2017_name,
        ).to_pandas_dataframe()
x_test_2017, y_test_2017 = test_2017.drop(labels=["Label"], axis=1), test_2017["Label"]

CPU times: user 8.76 s, sys: 4.88 s, total: 13.6 s
Wall time: 1min 11s


In [51]:
%%time
y_test_2017_predict = fitted_model.predict(x_test_2017)

{"message": "df6e07e1-03b9-4048-ad08-5548abb605ac - CacheDriver:Cached token is expired at 2021-01-30 23:09:23.734172.  Refreshing"}
{"message": "df6e07e1-03b9-4048-ad08-5548abb605ac - CacheDriver:Cached token is expired at 2021-01-30 23:09:23.734172.  Refreshing"}
{"message": "df6e07e1-03b9-4048-ad08-5548abb605ac - CacheDriver:Cached token is expired at 2021-01-30 23:09:23.734172.  Refreshing"}
{"message": "df6e07e1-03b9-4048-ad08-5548abb605ac - CacheDriver:Cached token is expired at 2021-01-30 23:09:23.734172.  Refreshing"}
{"message": "df6e07e1-03b9-4048-ad08-5548abb605ac - TokenRequest:Getting a new token from a refresh token"}
{"message": "df6e07e1-03b9-4048-ad08-5548abb605ac - TokenRequest:Getting a new token from a refresh token"}
{"message": "df6e07e1-03b9-4048-ad08-5548abb605ac - TokenRequest:Getting a new token from a refresh token"}
{"message": "df6e07e1-03b9-4048-ad08-5548abb605ac - TokenRequest:Getting a new token from a refresh token"}
{"message": "df6e07e1-03b9-4048-ad08

CPU times: user 42min 25s, sys: 2min 26s, total: 44min 51s
Wall time: 20min 31s


In [52]:
%%time
cr = classification_report(digits=4,
                           y_true=y_test_2017,
                           y_pred=y_test_2017_predict,
                           output_dict=False)
print(cr)

                          precision    recall  f1-score   support

                  Benign     0.8239    0.9143    0.8668    953268
                     Bot     0.0000    0.0000    0.0000      1956
        Brute Force -Web     0.0000    0.0000    0.0000      1507
        Brute Force -XSS     0.0000    0.0000    0.0000       635
        DDOS attack-HOIC     0.0000    0.0000    0.0000         0
   DoS attacks-GoldenEye     0.0336    0.0459    0.0388      7714
        DoS attacks-Hulk     0.0029    0.0001    0.0002    164013
DoS attacks-SlowHTTPTest     0.0000    0.0000    0.0000      2327
   DoS attacks-Slowloris     0.2037    0.9610    0.3361      4154
          FTP-BruteForce     0.0000    0.0000    0.0000      6443
           Infilteration     0.0000    0.0000    0.0000        33
           SQL Injection     0.0000    0.0000    0.0000        21
          SSH-Bruteforce     0.0054    0.0201    0.0085      5885

                accuracy                         0.7632   1147956
        

In [53]:
del test_2017
del x_test_2017

## Model Deployment

Register the best model into the workspace

In [37]:
%%time
model = package_utils.register_model(model_name="automl-jupyter",
                                     model_path="outputs/model.pkl",
                                     run=best_run)

CPU times: user 467 ms, sys: 4.68 ms, total: 472 ms
Wall time: 4.29 s


In [38]:
print(model.serialize())

{'createdTime': '2021-01-30T22:11:46.123640+00:00', 'createdBy': {'userObjectId': 'b0e0dfc5-cdf2-4c0f-b904-9072f8246826', 'userPuId': '1003200111DF7E2A', 'userIss': 'https://sts.windows.net/660b3398-b80e-49d2-bc5b-ac1dc93b5254/', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': 'ODL_User 136633'}, 'description': None, 'id': 'automl-jupyter:1', 'mimeType': 'application/json', 'name': 'automl-jupyter', 'framework': 'Custom', 'frameworkVersion': None, 'tags': {}, 'properties': {}, 'unpack': False, 'url': 'aml://asset/4a80bfa1c9ba494e924baa21aebcdcef', 'version': 1, 'experimentName': 'jupyter-2021-01-29-09-02', 'runId': 'AutoML_ae1d998c-ec87-447d-bf70-328476f80063_14', 'runDetails': 'Run(Experiment: jupyter-2021-01-29-09-02,\nId: AutoML_ae1d998c-ec87-447d-bf70-328476f80063_14,\nType: azureml.scriptrun,\nStatus: Completed)', 'datasets': [], 'resourceConfiguration': None, 'sampleInputDatasetId': None, 'sampleOutputDatasetId': None}


Deploy the registered model to an Azure Container instance

In [39]:
%%time
service = deploy_config.main(model_name="automl-jupyter",
                             deployment_name="automl-jupyter")

{"message": "main", "environment": "Environment(Name: automl-jupyter,\nVersion: None)"}
{"message": "main", "environment": "Environment(Name: automl-jupyter,\nVersion: None)"}
{"message": "main", "environment": "Environment(Name: automl-jupyter,\nVersion: None)"}
{"message": "main", "environment": "Environment(Name: automl-jupyter,\nVersion: None)"}
{"message": "main", "inference_config": "InferenceConfig(entry_script=model/deploy/score.py, runtime=None, conda_file=None, extra_docker_file_steps=None, source_directory=/app/nd00333, enable_gpu=None, base_image=None, base_image_registry=<azureml.core.container_registry.ContainerRegistry object at 0x7f68209bf290>)"}
{"message": "main", "inference_config": "InferenceConfig(entry_script=model/deploy/score.py, runtime=None, conda_file=None, extra_docker_file_steps=None, source_directory=/app/nd00333, enable_gpu=None, base_image=None, base_image_registry=<azureml.core.container_registry.ContainerRegistry object at 0x7f68209bf290>)"}
{"message"

CPU times: user 2.12 s, sys: 169 ms, total: 2.29 s
Wall time: 13 s


In [40]:
%%time
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: user 4.29 s, sys: 360 ms, total: 4.65 s
Wall time: 3min 14s


In [41]:
assert service.state == "Healthy"

Test the service endpoint

Fetch the API keys of the service endpoint

In [42]:
primary_api_key, secondary_api_key = service.get_keys()

Retrive the scoring url of the service endpoint

In [43]:
url = service.scoring_uri
print(url)

http://fede3df5-b8eb-493f-ad37-45f325532bc0.southcentralus.azurecontainer.io/score


Prepare a subset of the test dataset for submission to the service

In [44]:
input_data = json.dumps({'data': x_test[0:1].to_dict(orient='records')})
with open("data.json", "w") as _f:
    _f.write(input_data)
!cat data.json

{"data": [{"Flow Duration": 3761843, "TotLen Fwd Pkts": 1441, "TotLen Bwd Pkts": 1731, "Fwd Pkt Len Std": 191, "Bwd Pkt Len Max": 1179, "Bwd Pkt Len Std": 405, "Flow Byts/s": 843, "Flow Pkts/s": 6, "Flow IAT Max": 953181, "Bwd IAT Min": 124510, "Bwd Header Len": 172, "Pkt Len Max": 1179, "Pkt Len Std": 279, "RST Flag Cnt": 1, "PSH Flag Cnt": 1, "ECE Flag Cnt": 1, "Init Fwd Win Byts": 8192, "Init Bwd Win Byts": 62644, "Fwd Seg Size Min": 20}]}

Call the service using the input_data

In [45]:
print(service.run(input_data))

{"result": ["Benign"]}


Send a post request to the service endpoint using curl

In [46]:
%%time
!curl -X POST \
      -H 'Content-Type: application/json' \
      -H "Authorization: Bearer $secondary_api_key" \
      --data @data.json $url

"{\"result\": [\"Benign\"]}"CPU times: user 31.4 ms, sys: 34.4 ms, total: 65.7 ms
Wall time: 1.56 s


Send a post request to the service endpoint programatically

In [47]:
# Set the content type
headers = {"Content-Type": "application/json"}
# If authentication is enabled, set the authorization header
headers["Authorization"] = f"Bearer {secondary_api_key}"

resp = requests.post(url, input_data, headers=headers)
print(resp.json())

{"result": ["Benign"]}


In [49]:
del x_test

Print the service logs

In [54]:
print(service.get_logs())

2021-01-30T22:14:51,530708557+00:00 - nginx/run 
2021-01-30T22:14:51,535531061+00:00 - gunicorn/run 
2021-01-30T22:14:51,536593362+00:00 - iot-server/run 
2021-01-30T22:14:51,539587765+00:00 - rsyslog/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-01-30T22:14:51,755418137+00:00 - iot-server/finish 1 0
2021-01-30T22:14:51,756937239+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (15)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 42
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-01-30 22:14:52,598 | root | INFO | Starting up app insights client
2021-01-30 22:14:52,598 | root | INFO | Starting up request id generator
2021-01-30 22:14:52,598 | root | INFO | Starting up app insight hooks
2021-01-30 22:14:52,598 | root | INFO | Invoking user's init function
2021-01-30 22:14:55,232 | root | INFO | Users's init has completed suc

Delete the service endpoint

In [55]:
service.delete()

Delete the compute cluster

In [56]:
cluster_name = package_utils.trim_cluster_name(workspace.name)
print(f"cluster_name {cluster_name}")

cluster_name starts-ws-136633


In [59]:
try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    compute_target.delete()
    print(f"compute_target {compute_target} is being deleted")    
except ComputeTargetException:
    print(f"compute_target {cluster_name} does not exist")

compute_target starts-ws-136633 does not exist
